In [1]:
import sys
sys.path.append('../')
from setting import config_read

In [2]:
import numpy as np
import itertools
from owlready2 import get_ontology
from elasticsearch import Elasticsearch
from konlpy.tag import Okt

* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation




In [3]:
# Read config file
config = config_read('../')

data_path = config['owl']['path']
onto = get_ontology(data_path).load()

# Elasticsearch
server_ip = config['elasticsearch']['ip']
index_name = config['elasticsearch']['name']
es = Elasticsearch(server_ip)

# OKT
okt = Okt()

In [4]:
user_input = input("Question: ")
query_pos = okt.pos(user_input, norm=True)
query_terms = [q[0] for q in query_pos if q[1] not in ["Josa", "Punctuation"]]
query_terms

['용감한', '형제', '작곡', '노래', '장르']

In [5]:
query_patitions = [[" ".join(query_terms)]]

for slice_num in range(1,len(query_terms)):
    for slice_list in itertools.combinations(range(len(query_terms)-1), slice_num):
        qp = []
        qp.append(" ".join(query_terms[:slice_list[0]+1]))
        
        for s_i, slice_idx in enumerate(slice_list):
            if s_i == len(slice_list) - 1 : break
            qp.append(" ".join(query_terms[slice_idx+1:slice_list[s_i+1]+1]))

        qp.append(" ".join(query_terms[slice_list[-1]+1:]))
        query_patitions.append(qp)
    
query_patitions

[['용감한 형제 작곡 노래 장르'],
 ['용감한', '형제 작곡 노래 장르'],
 ['용감한 형제', '작곡 노래 장르'],
 ['용감한 형제 작곡', '노래 장르'],
 ['용감한 형제 작곡 노래', '장르'],
 ['용감한', '형제', '작곡 노래 장르'],
 ['용감한', '형제 작곡', '노래 장르'],
 ['용감한', '형제 작곡 노래', '장르'],
 ['용감한 형제', '작곡', '노래 장르'],
 ['용감한 형제', '작곡 노래', '장르'],
 ['용감한 형제 작곡', '노래', '장르'],
 ['용감한', '형제', '작곡', '노래 장르'],
 ['용감한', '형제', '작곡 노래', '장르'],
 ['용감한', '형제 작곡', '노래', '장르'],
 ['용감한 형제', '작곡', '노래', '장르'],
 ['용감한', '형제', '작곡', '노래', '장르']]

In [6]:
score_list = []

for qp in query_patitions:
    score = 0
    for q in qp:
        search_query = {"query":{"term":{"Annotation Values.keyword": q}}}
        result = es.search(index=index_name, body=search_query)
        if result['hits']['max_score'] :
            score += result['hits']['max_score']
    score_list.append(score/len(qp))

p_star = query_patitions[np.argmax(score_list)]

print(p_star)
print(score_list)

['용감한 형제', '작곡', '노래', '장르']
[0.0, 0.0, 3.58557035, 0.0, 3.2757437, 0.0, 0.0, 2.1838291333333335, 4.780760466666666, 4.5742093666666666, 4.5742093666666666, 1.792785175, 1.63787185, 3.430657025, 7.016227375, 4.17875376]


In [7]:
candidates_list = []

for token in p_star:
    search_query = {"query":{"term":{"Annotation Values.keyword": token}}}
    result = es.search(index=index_name, body=search_query)
    candidates = []
    for res in result['hits']['hits']:
        candidates.append(res['_source']['URI'])
    candidates_list.append(candidates)

resource_combinations = list(itertools.product(*candidates_list))
resource_combinations

[('skmo:용감한_형제', 'skmo:isComposedBy', 'mo:Track', 'skmo:Genre'),
 ('skmo:용감한_형제', 'skmo:isComposedBy', 'mo:Track', 'skmo:hasGenre')]